In [ ]:
# Run this cell in a folder contained by the current project
using Pkg
#Pkg.activate()                 # Walks up the path tree until finds Project.toml
#Pkg.activate(".")              # 
#Pkg.activate("@__DIR__")       # 
Pkg.activate("../")            #
Pkg.instantiate()

In [ ]:
using Plots
using LaTeXStrings
using Graphs
using GraphPlot
using Karnak
using NetworkLayout
using Colors
using Random
using LinearAlgebra
using OffsetArrays
using SparseArrays
using Serialization
using IterativeSolvers
using Krylov
using BenchmarkTools
using Profile
using ProfileView
using Statistics
using DifferentialEquations
using LoopVectorization
using Folds
using Transducers
using Distributed
using JLD2
using OnlineStats
using ImageFiltering
using SavitzkyGolay
using Interpolations
using DataInterpolations
using Polynomials
using CurveFit
using DataFrames
using LsqFit
using ForwardDiff

In [ ]:
function enum!(d,k)
    return get!(d,k,length(d)+1)
end

### Test

In [ ]:
mutable struct RunningStat
    n::Int
    m::Real # mean
    s::Real # std
end

function RunningStat()
    return RunningStat(0,0.0,0.0)
end

function Base.push!(rs::RunningStat,v::Real)
    if rs.n>0
        rs.n += 1
        new_m = rs.m + (v-rs.m)/rs.n
        rs.s += (v-rs.m)*(v-new_m)
        rs.m = new_m
    else
        rs.n = 1
        rs.m = v
        rs.s = 0
    end
    return rs.m
end

function Statistics.mean(rs::RunningStat)
    return rs.m 
end

function Statistics.var(rs::RunningStat)
    return rs.s/(rs.n-1)
end

function Statistics.std(rs::RunningStat)
    return sqrt(var(rs))
end

### Tests

In [ ]:
list_v = 10 .+ 5.0*randn(1000)
;

In [ ]:
rs = RunningStat()

In [ ]:
for v in list_v
    push!(rs,v)
end

In [ ]:
mean(rs),std(rs)

In [ ]:
mean(list_v),std(list_v)

In [ ]:
function R_squared(y_data,y_fit)
    y_mean = mean(y_data)
    SS_res = sum((y_data .- y_fit) .^ 2)
    SS_tot = sum((y_data .- y_mean) .^ 2)
    R_sqr = 1 - SS_res / SS_tot
    return R_sqr
end

### Tests

In [ ]:
# Define the model function
@. model(x, p) = p[1] * x + p[2]

# Sample data (example)
x_data = [1.0, 2.0, 3.0, 4.0, 5.0]
y_data = [2.2, 2.8, 3.6, 4.5, 5.1]

# Initial guess for parameters
p0 = [1.0, 1.0]

# Perform curve fitting
fit = LsqFit.curve_fit(model, x_data, y_data, p0)

# Compute fitted values
y_fit = model(x_data, coef(fit))

# Compute R_sqr
R_sqr = R_squared(y_data,y_fit)

In [ ]:
plot()
#plot!(yscale=:log10)
#plot!(ylim=(0.0,1.0))
plot!(xlabel=L"x")
plot!(ylabel=L"y")
scatter!(x_data,y_data,label="data")
plot!(x_data,y_fit,label="fit, \$R^2=0.995\$")
#plot!(0.0:0.0,x->0.0,label="\$z=1\$",width=0.0,c=:white)
#plot!(legendtitle=L"\mathrm{1D},\;z=1",legendtitlefontsize=8)
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(
#    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
#    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
#    tickfont=font(11),
#)

In [ ]:
x0 = 1.0
A = 1.0
B = 3.0
C = 0.0
p0 = [x0,A,B,C] # p1=x0, p2=A, p3=B, p4=C
@. sigmoid(x,p)  = p[2]/(1+exp(-p[3]*(x-p[1]))) # sigmoid
@. dsigmoid(x,p) = p[2]*p[3]*exp(-p[3]*(x-p[1]))/(1+exp(-p[3]*(x-p[1])))^2 # derivative of sigmoid
@. psigmoid(x,p) = p[2]/p[3]*log(1+exp(p[3]*(x-p[1])))+p[4] # primitive of sigmoid

### Test

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"f(x)",
)
plot!(-2:0.01:4.0,x->sigmoid(x,p0),label="")

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"df/dx",    
)
plot!(-2:0.01:4.0,x->dsigmoid(x,p0),label="")
scatter!([x0],[A*B/4],label="")

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"F(x)",
)
plot!(-2:0.01:4.0,x->psigmoid(x,p0),label="")

# Load networks from raw data and save them in JLD2 format

In [ ]:
list_nets_dat = """../redes/Italy/red_crossing.dat
../redes/Italy/red_counterattack.dat
../redes/Italy/red_pressure_loss.dat
../redes/Italy/red_T_build_up.dat
../redes/Italy/red_direct_play.dat
../redes/Italy/red_pressure_point.dat
../redes/Italy/red_shots.dat
../redes/Italy/red_flow_rate.dat
../redes/Italy/red_T_maintenance.dat
../redes/Italy/red_T_zona_media.dat
../redes/Germany/red_crossing.dat
../redes/Germany/red_counterattack.dat
../redes/Germany/red_pressure_loss.dat
../redes/Germany/red_T_build_up.dat
../redes/Germany/red_direct_play.dat
../redes/Germany/red_pressure_point.dat
../redes/Germany/red_shots.dat
../redes/Germany/red_flow_rate.dat
../redes/Germany/red_T_maintenance.dat
../redes/Germany/red_T_zona_media.dat
../redes/Spain/red_crossing.dat
../redes/Spain/red_counterattack.dat
../redes/Spain/red_pressure_loss.dat
../redes/Spain/red_T_build_up.dat
../redes/Spain/red_direct_play.dat
../redes/Spain/red_pressure_point.dat
../redes/Spain/red_shots.dat
../redes/Spain/red_flow_rate.dat
../redes/Spain/red_T_maintenance.dat
../redes/Spain/red_T_zona_media.dat
../redes/England/red_crossing.dat
../redes/England/red_counterattack.dat
../redes/England/red_pressure_loss.dat
../redes/England/red_T_build_up.dat
../redes/England/red_direct_play.dat
../redes/England/red_pressure_point.dat
../redes/England/red_shots.dat
../redes/England/red_flow_rate.dat
../redes/England/red_T_maintenance.dat
../redes/England/red_T_zona_media.dat
../redes/France/red_crossing.dat
../redes/France/red_counterattack.dat
../redes/France/red_pressure_loss.dat
../redes/France/red_T_build_up.dat
../redes/France/red_direct_play.dat
../redes/France/red_pressure_point.dat
../redes/France/red_shots.dat
../redes/France/red_flow_rate.dat
../redes/France/red_T_maintenance.dat
../redes/France/red_T_zona_media.dat"""

In [ ]:
dict_vecs_i_j_phi_jld2 = Dict()
for archivo in split(list_nets_dat)
    println(archivo)
    vec_i = Vector{Int64}()
    vec_j = Vector{Int64}()
    vec_phi_ij = Vector{Float64}()    
    open(archivo) do fh
        for line in readlines(fh)[2:end]
            cols=split(replace(line,"," => " "))
            #println(cols)
            push!(vec_i,parse(Int64,cols[1]))
            push!(vec_j,parse(Int64,cols[2]))
            push!(vec_phi_ij,parse(Float64,cols[3]))
        end
    end
    tag = replace(archivo, "../redes/" => "")
    tag = replace(tag, ".dat" => "")
    tag = replace(tag, "red_" => "")
    tag = replace(tag, "/" => " ")
    tag = split(tag)
    tag = (tag[1],tag[2])
    dict_vecs_i_j_phi_jld2[tag] = (vec_i,vec_j,vec_phi_ij)
end

In [ ]:
redes_jld2

In [ ]:
@save "../redes/dict_vecs_i_j_phi.jld2" dict_vecs_i_j_phi_jld2

In [ ]:
@load "../redes/dict_vecs_i_j_phi.jld2" dict_vecs_i_j_phi_jld2

In [ ]:
dict_vecs_i_j_phi_jld2

In [ ]:
dict_vecs_i_j_phi_jld2[("France","crossing")]

# References

[1] ...